In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\camilo\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag']]

In [6]:
y = df[["koi_disposition"]]

In [7]:
y = y.values.reshape(-1,)
y.shape

(6991,)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.20)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
457,0,0,0,0,6.100276,135.586560,0.015,4.7476,766.6,2.59,990,227.15,43.2,1,5957,4.506,0.937,293.18454,37.969841,15.678
57,0,0,0,0,43.844397,151.935380,0.852,6.8020,615.2,2.87,531,18.77,83.7,2,5854,4.406,1.012,300.86398,44.337551,12.882
2630,0,0,0,0,14.282281,180.846770,0.019,2.6588,983.2,1.82,446,9.38,44.7,1,4102,4.675,0.595,284.47849,40.933102,14.672
1424,0,0,0,0,56.259088,164.219930,0.026,8.1140,348.8,2.22,543,20.49,29.7,5,6038,4.284,1.191,291.11188,39.949108,14.247
2150,0,0,0,0,11.872908,180.895933,0.904,2.5342,332.7,2.25,863,131.55,84.7,1,6147,4.412,1.040,286.68945,39.211910,11.072


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=3, activation='softmax'))

In [13]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [14]:
model.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=2)

Train on 5592 samples
Epoch 1/100
5592/5592 - 1s - loss: 0.7117 - accuracy: 0.6586
Epoch 2/100
5592/5592 - 0s - loss: 0.4221 - accuracy: 0.7883
Epoch 3/100
5592/5592 - 0s - loss: 0.3850 - accuracy: 0.7960
Epoch 4/100
5592/5592 - 0s - loss: 0.3762 - accuracy: 0.8006
Epoch 5/100
5592/5592 - 0s - loss: 0.3709 - accuracy: 0.8033
Epoch 6/100
5592/5592 - 0s - loss: 0.3690 - accuracy: 0.8008
Epoch 7/100
5592/5592 - 0s - loss: 0.3658 - accuracy: 0.8065
Epoch 8/100
5592/5592 - 0s - loss: 0.3637 - accuracy: 0.8104
Epoch 9/100
5592/5592 - 0s - loss: 0.3625 - accuracy: 0.8067
Epoch 10/100
5592/5592 - 0s - loss: 0.3606 - accuracy: 0.8078
Epoch 11/100
5592/5592 - 0s - loss: 0.3589 - accuracy: 0.8090
Epoch 12/100
5592/5592 - 0s - loss: 0.3588 - accuracy: 0.8101
Epoch 13/100
5592/5592 - 0s - loss: 0.3558 - accuracy: 0.8101
Epoch 14/100
5592/5592 - 0s - loss: 0.3575 - accuracy: 0.8124
Epoch 15/100
5592/5592 - 0s - loss: 0.3544 - accuracy: 0.8137
Epoch 16/100
5592/5592 - 0s - loss: 0.3537 - accuracy: 0.

In [15]:
print(f"Training Data Score: {model.evaluate(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {model.evaluate(X_test_scaled, y_test_categorical)}")

5592/5592 [==============================] - 0s 33us/sample - loss: 0.2966 - accuracy: 0.8518
Training Data Score: [0.2966137392838114, 0.8517525]
1399/1399 [==============================] - 0s 31us/sample - loss: 0.3550 - accuracy: 0.8256
Testing Data Score: [0.3549640427068611, 0.8255897]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [16]:
list = [25, 50, 75, 100]

for x in list:
    model2 = Sequential()

    model2.add(Dense(units=x, activation='relu', input_dim=20))
    model2.add(Dense(units=3, activation='softmax'))
    
    model2.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
    
    model2.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=0)
    
    print(f"Units: {x}, testing score:{model2.evaluate(X_test_scaled, y_test_categorical)}")

1399/1399 [==============================] - 0s 111us/sample - loss: 0.3642 - accuracy: 0.8134
Units: 25, testing score:[0.36419131450775777, 0.8134382]
1399/1399 [==============================] - 0s 69us/sample - loss: 0.3406 - accuracy: 0.8299
Units: 50, testing score:[0.34059211688437063, 0.8298785]
1399/1399 [==============================] - 0s 66us/sample - loss: 0.3502 - accuracy: 0.8234
Units: 75, testing score:[0.35021150421039643, 0.8234453]
1399/1399 [==============================] - 0s 70us/sample - loss: 0.3499 - accuracy: 0.8270
Units: 100, testing score:[0.3499116282151204, 0.8270193]


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)